In [73]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests
import torch

import pandas as pd
import numpy as np
import time

In [91]:
df_images = pd.read_csv("data/silver_20250322_Airline_Reviews_Resnet50.csv")
df_images

,Unnamed: 0,RowId,Airline Name,Review_Title,Review Date,Top Review Image Url,img_resnet50_score,img_resnet50_label
0,0,0,AB Aviation,"""pretty decent airline""",11th November 2019,https://www.airlinequality.com/wp-content/uplo...,5.190325,military aircraft
1,13,13,Adria Airways,"""Would not fly again""",30th June 2018,https://www.airlinequality.com/wp-content/uplo...,6.628997,mail bag
2,108,108,Aegean Airlines,“claims to be pet-friendly”,27th October 2024,https://www.airlinequality.com/wp-content/uplo...,5.040598,jeep
3,109,109,Aegean Airlines,“Great value for money!”,13th October 2024,https://www.airlinequality.com/wp-content/uplo...,7.355824,espresso
4,110,110,Aegean Airlines,"""75 euro for 1 10 kg bag round trip""",4th October 2024,https://www.airlinequality.com/wp-content/uplo...,5.758941,desk
...,...,...,...,...,...,...,...,...
1892,24298,24298,ZIPAIR,"""being periodically harassed and ridiculed""",19th September 2023,https://www.airlinequality.com/wp-content/uplo...,4.510304,photocopier
1893,24304,24304,ZIPAIR,"""can highly recommend ZIPAIR""",23rd July 2023,https://www.airlinequality.com/wp-content/uplo...,4.862087,waffle iron
1894,24312,24312,ZIPAIR,"""I heard back 3 months later""",18th May 2023,https://www.airlinequality.com/wp-content/uplo...,5.286357,assault rifle
1895,24320,24320,ZIPAIR,ZIPAIR customer review,21st April 2023,https://www.airlinequality.com/wp-content/uplo...,4.154579,plate


#### Generate BLIP image captions

In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [101]:
def blip_image_caption(img_url, row_id=None):
    # Load an image from URL
    url = img_url
    print(row_id, img_url)
    
    image = Image.open(requests.get(url, stream=True).raw)
    
    # Generate caption
    inputs = processor(images=image, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.generate(**inputs)
        
    caption = processor.decode(outputs[0], skip_special_tokens=True)
    print(caption)
    
    #1-second delay so as not to spam/DOS the source webserver
    time.sleep(1)

    return caption

In [103]:
blip_image_caption('https://www.airlinequality.com/wp-content/uploads/2018/06/IMG_1571-1-500x500.jpg', 13)

13 https://www.airlinequality.com/wp-content/uploads/2018/06/IMG_1571-1-500x500.jpg
the green leather case is attached to the back of the suitcase


'the green leather case is attached to the back of the suitcase'

In [105]:
df_images[['img_blip_caption']] = df_images.apply(
    lambda row: pd.Series(blip_image_caption(row['Top Review Image Url'], row['RowId'])), axis=1
)

0 https://www.airlinequality.com/wp-content/uploads/2019/11/20191102_123305-500x500.jpg
a man taking a picture of a plane
13 https://www.airlinequality.com/wp-content/uploads/2018/06/IMG_1571-1-500x500.jpg
the green leather case is attached to the back of the suitcase
108 https://www.airlinequality.com/wp-content/uploads/2024/10/IMG_5873-500x500.jpeg
a woman holding a dog in a cage
109 https://www.airlinequality.com/wp-content/uploads/2024/10/20241008_FRA-ATH-A3831_espresso-500x500.jpg
a cup of coffee on a table
110 https://www.airlinequality.com/wp-content/uploads/2024/10/PXL_20241004_073123468-500x500.jpg
a computer desk with a monitor and a printer
118 https://www.airlinequality.com/wp-content/uploads/2024/07/inbound8469499039478717598-500x500.jpg
a tray with food and drinks on it
130 https://www.airlinequality.com/wp-content/uploads/2024/05/IMG_20240518_103907_edit_2247906755096047-500x500.jpg
the seats in the first class of the airbus 777
135 https://www.airlinequality.com/wp-cont

#### Persist to csv file

In [108]:
df_images[['RowId','Airline Name','Review_Title','Review Date','Top Review Image Url','img_resnet50_score',\
           'img_resnet50_label','img_blip_caption']].to_csv("data/silver_20250322_Airline_Reviews_ImageCaption.csv")